In [ ]:
import numpy as np
import MDAnalysis as mda
import sys
from MDAnalysis.analysis.distances import distance_array
import networkx as nx
import pandas as pd
import mdtraj as md
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("TkAgg")
import itertools
import time

In [ ]:
# Queste sono da importare direttamente dal pdb ma ce da risolvere la numerazione

amino_dict_num = {0:'TYR_1', 1:'THR_2', 2:'ILE_3', 3:'ALA_4', 4:'ALA_5', 5:'LEU_6', 6:'LEU_7', 7:'SER_8', 8:'PRO_9', 9:'TYR_10', 10:'SER_11'}
pdb_resids = list(range(0, 11))

#ref_structure = fibril.pdb
#traj = start_clust-dt10000.xtc
Min_Distance = 4

In [ ]:
def contact_extraction(contact_list, contacts_histogram, pdb_resids):
    # Function to extract the informations from the pairs list
    for an in pdb_resids:
        # pdb_resids is the aminocid number
        # Selection of the rows contaning the aminoacid number of interest.
        # Note the | and not the &
        is_an = (contact_list['ai_resnumber'] == an) | (contact_list['aj_resnumber'] == an)

        # Filtering a dataframe containing only the aminoacid selected.
        contacts_an = contact_list[is_an]
        #print(pairs_an.to_string())

        print(f'\nTotal amount of contacts made by {amino_dict_num.get(an)}:', len(contacts_an))    
        contacts_an.loc[contacts_an['ai_resnumber'] == an, 'an_with_who'] = contacts_an['aj_resname']# + '_' + contacts_an['aj_resnumber'].astype(str)
        contacts_an.loc[contacts_an['ai_resnumber'] != an, 'an_with_who'] = contacts_an['ai_resname']# + '_' + contacts_an['ai_resnumber'].astype(str)    
        #print(pairs_an.to_string())
        
        # Create Pandas Series with contact counts for every aminoacid in an selection:
        # an_with_who count of the unique values in column "an_with_who" in pairs_an dataframe
        an_with_who = contacts_an.an_with_who.value_counts()
        print(f'\nList of contacts based on aminoacid:\n{an_with_who.to_string()}')

        # Change "an_with_who" in a dictionary to put in a matrix
        contacts_histogram['{0}'.format(an)] = an_with_who.to_dict()
        #print(f'\nAnd the consequent nested dictionary:\n{contacts_histogram}\n')
    
    return contacts_histogram#, sigma_histogram

def make_contacts_histo_df(contacts_histogram):
    # Contact dataframe preparation
    histo_df = pd.DataFrame(contacts_histogram)
    histo_df[''] = histo_df.index.astype(str)
    histo_df[''] = histo_df[''].str.split('_').str[1].astype(int)
    histo_df.sort_values(by = [''], inplace = True)
    histo_df.drop(columns='', inplace=True)
    histo_df = histo_df.fillna(0)
    histo_df.columns = list(amino_dict_num.values())
    #print(histo_df.to_string())
    return histo_df

In [ ]:
#structure parameters

#topology = md.load(ref_structure).topology
topology = md.load('fibril.pdb').topology
#trajectory = md.load(traj, top=ref_structure)
trajectory = md.load('start_clust-dt10000.xtc', top='fibril.pdb')
frames=trajectory.n_frames				#Number of frames da provare a vedere cosa viene fuori con MDAnalysis
chains=topology.n_chains				#Number of chains
atoms=int(topology.n_atoms/chains)			#Number of atoms in each monomer 
AminoAcids = int(topology.n_residues/chains)#-2		#Number of residues per chain ('-2' avoid the N- and C- cap residues as individual residues)


print(f'Number of chains in the system: {chains}\nNumber of atoms in each monomer:{atoms}\nNumber of aminoacids in every chain:{AminoAcids}')

isum=1
atoms_list, atomsperAminoAcid, residue_list=[], [], []
for residue in topology.chain(0).residues:
    atoms_list.append(residue.n_atoms)
    residue_list.append(residue)
    ', '.join(map(lambda x: "'" + x + "'", str(residue_list)))
print(f'{len(residue_list)} residues in chain:{residue_list}')

#The N- and C- cap residues are part of the 1st and last residue index. If no N- and C- cap residues for the protein, comment the line below using "#"
#del residue_list[0]; del residue_list[-1] 

for ii in range(len(atoms_list)):
    isum = isum + atoms_list[ii]
    atomsperAminoAcid.append(isum)
atomsperAminoAcid.insert(0, 1)

#print('atomsperAminoAcid', atomsperAminoAcid) # Questo e' perche' le chains partono da 0 e i gruppi vengono ottenuti moltiplicando per il resid

#The N- and C- cap residues are part of the 1st and last residue index. If no N- and C- cap residues for the protein, comment the line below using "#"
#del atomsperAminoAcid[1]; del atomsperAminoAcid[-2]      

# Create Universe
print('Create Universe')
uni = mda.Universe('fibril.pdb','start_clust-dt10000.xtc')
n,t = list(enumerate(uni.trajectory))[0]
box = t.dimensions[:6]
print('Box dimensions', box)

atom_Groups = [[] for x in range(chains)]
m_start=0
for m in range(0,chains):
    # With this functions every chain is defined as a group of atoms
    # m is the chain number, atom is the number of atoms in every chain
    m_end = atoms * (m+1)
    atom_Groups[m].extend([uni.select_atoms('bynum '+ str(m_start) + ':' + str(m_end))])
    m_start = m_end + 1

AtomGroups = [[] for x in range(chains)]
for m in range(0,chains):
    for aa in range(0,AminoAcids):
        AtomGroups[m].extend([uni.select_atoms('bynum '+str(atoms*m + atomsperAminoAcid[aa])+':'+str(atoms*m + atomsperAminoAcid[aa + 1] - 1 ))])

In [ ]:
cluster_for_traj, chain_resid_traj, contacts_traj, norm_contacts_traj = {}, {}, {}, {}
for tt in uni.trajectory:
    '''
    Analysis of chains interactions in every frame of trajectory.
    In every frame it is created a Dataframe containing a list of
    interactions among the chains involved in a fibril (less than 4A)
    '''

    start_time = time.time()
    print(f'\n\nStarting frame {tt.frame} analysis out of {len(uni.trajectory)} total frames')
    
    # Creation of an empty dataframe which will be filled with a list of 
    # interactions between chains less than 4A far

    chain_list = []
    for i in range(chains):
        # Here a combinations of chains is made for every chain in the system
        chain_list.append(i)
    chain_comb = list(itertools.combinations(chain_list, 2))


    df_chain_ai, df_chain_aj, chain_distance = [], [], []
    for n in range(0, len(chain_comb)):
        # Here the combination list of tuple is rearranged in two lists
        # Only the chains within 4A cutoff are inserted into the dataframe
        i = chain_comb[n][0]
        j = chain_comb[n][1]
        dist = distance_array(atom_Groups[i][0].positions,atom_Groups[j][0].positions,box).min()
        if dist <= Min_Distance:
            df_chain_ai.append(i)
            df_chain_aj.append(j)
            chain_distance.append(dist)
    
    # The following DataFrame contains all the interactions among the chains involved in fibril
    chains_interactions = pd.DataFrame(columns=['chain_ai', 'chain_aj', 'min_distance'])
    chains_interactions['chain_ai'] = df_chain_ai
    chains_interactions['chain_aj'] = df_chain_aj
    chains_interactions['min_distance'] = chain_distance
    
    
    chains_in_fibril = list((set(chains_interactions['chain_ai'])) | (set(chains_interactions['chain_aj'])))
    chains_in_fibril.sort()
    print("--- %s seconds ---" % (time.time() - start_time))
    print('Number of chains interactions:', len(chains_interactions))
    print(f'{len(chains_in_fibril)} chains in fibril:{chains_in_fibril}')

    # Here a DataFrame of the interacting aminoacids is made for every contacting chain
    trj_amino_interactions = []
    for i, j in zip(df_chain_ai, df_chain_aj):
        # For every pair of chains
        en_i, atoms_i, ai_resnumber = [], [], []
        for e, a in enumerate(AtomGroups[int(float(i))]):
            # Select the AtomGroup of the wanted chain (i)
            # Append the residue number of the chain + the chain number
            en_i.append(str(e) + '_' + str(i))
            # Append the residue number alone
            ai_resnumber.append(e)
            # Append the corresponding AtomGroups of the chain
            atoms_i.append(a)
        en_j, atoms_j, aj_resnumber = [], [], []
        for e, a in enumerate(AtomGroups[int(float(j))]):
            # Select the AtomGroup of the wanted chain (i)
            # Append the residue number of the chain + the chain number
            en_j.append(str(e) + '_' + str(j))
            # Append the residue number alone
            aj_resnumber.append(e)
            # Append the corresponding AtomGroups of the chain
            atoms_j.append(a)

        # Those three lists have the same order, so combination will have the same shape
        # 11 aminoacids per chain -> 121 combinations
        en_comb = list(itertools.product(en_i, en_j)) # len 121
        resnumber_comb = list(itertools.product(ai_resnumber, aj_resnumber))
        atoms_comb = list(itertools.product(atoms_i, atoms_j))

        amino_distance = []
        for atoms in atoms_comb:            
            amino_distance.append(distance_array(atoms[0].positions,atoms[1].positions,box).min())

        # Build the consequent dataframe
        amino_interactions = pd.DataFrame(columns=['resid_chain_i', 'resid_chain_j', 'ai_resnumber', 'aj_resnumber', 'ai_resname', 'aj_resname', 'amino_distance'])
        amino_interactions['resid_chain_i'] = [x[0] for x in en_comb]
        amino_interactions['resid_chain_j'] = [x[1] for x in en_comb]
        amino_interactions['ai_resnumber'] = [x[0] for x in resnumber_comb]
        amino_interactions['aj_resnumber'] = [x[1] for x in resnumber_comb]
        amino_interactions['ai_resname'] = amino_interactions['ai_resnumber'].map(amino_dict_num)
        amino_interactions['aj_resname'] = amino_interactions['aj_resnumber'].map(amino_dict_num)
        amino_interactions['amino_distance'] = amino_distance
        # Append as a list of DataFrames
        trj_amino_interactions.append(amino_interactions)

    # Merge the Dataframes in a single one
    trj_amino_interactions = pd.concat(trj_amino_interactions, ignore_index=True)
    # Keep only the contacts below the cutoff of 4A
    trj_amino_interactions = trj_amino_interactions.loc[trj_amino_interactions['amino_distance'] < Min_Distance]
    
    # Creation of a contact map for every interacting aminoacid in a fibril
    contacts_histogram = {}
    frame_contacts_histogram = contact_extraction(trj_amino_interactions, contacts_histogram, pdb_resids) # Questo pdb resids alla fine e' un elenco degli aminoacidi da mettere con residue_list che hai gia'
    frame_contacts = make_contacts_histo_df(frame_contacts_histogram)

    # Creation of the same contact matrix but with normalized values
    contacts_norm = frame_contacts.div(frame_contacts.max().max())
    
    
    # Save the DataFrames in a dictionary where the key is the number of frame
    # and the value are the Dataframes
    cluster_for_traj[tt.frame] = chains_interactions
    chain_resid_traj[tt.frame] = trj_amino_interactions
    contacts_traj[tt.frame] = frame_contacts
    norm_contacts_traj[tt.frame] = contacts_norm

#print(norm_contacts_traj)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_columns = None
for k in contacts_traj:
    print(f'Frame {k} out of {len(contacts_traj)} \n{contacts_traj[k]}\n\n\n\n')

In [ ]:
axn = len(contacts_traj)
fig, ax = plt.subplots(ncols=axn, figsize=(30,30))

for contacts, a in zip(contacts_traj.values(), list(range(0, axn))):
    contact_matrix = contacts.to_numpy()
    contact_matrix = contact_matrix.round(decimals=2)
    #print(contact_matrix)
  
    ax[a].imshow(contact_matrix, interpolation='nearest', cmap='Blues')
    ax[a].set_xticks(np.arange(len(contacts.columns)))
    ax[a].set_yticks(np.arange(len(contacts.index)))
    ax[a].set_xticklabels(list(contacts.columns))
    ax[a].set_yticklabels(list(contacts.index)) 
    ax[a].xaxis.tick_top()
    ax[a].set_title(f'Frame {a}', fontsize = 25, y = 1.14)
    plt.setp(ax[a].get_xticklabels(), rotation = 90, ha='center', rotation_mode='default')

    # Loop over data dimensions and create tet annotations
    for i in range(len(contacts.index)):
        for j in range(len(contacts.columns)):
            text = ax[a].text(j,i, contact_matrix[i,j], ha='center', va='center', color='w', fontsize = 5)
    fig.tight_layout()
fig.show()

In [ ]:
pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_columns = None
for k in norm_contacts_traj:
    print(f'Frame {k} out of {len(norm_contacts_traj)} \n{norm_contacts_traj[k]}\n\n\n\n')

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker
from matplotlib.colors import ListedColormap


axn = len(norm_contacts_traj)
fig, ax = plt.subplots(ncols=axn, figsize=(30,30))

for contacts, a in zip(norm_contacts_traj.values(), list(range(0, axn))):
    contact_matrix = contacts.to_numpy()
    contact_matrix = contact_matrix.round(decimals=2)
    print(contact_matrix)
  
    ax[a].imshow(contact_matrix, interpolation='nearest', cmap='Blues')
    ax[a].set_xticks(np.arange(len(contacts.columns)))
    ax[a].set_yticks(np.arange(len(contacts.index)))
    ax[a].set_xticklabels(list(contacts.columns))
    ax[a].set_yticklabels(list(contacts.index)) 
    ax[a].xaxis.tick_top()
    ax[a].set_title(f'Frame {a}', fontsize = 25, y = 1.14)
    plt.setp(ax[a].get_xticklabels(), rotation = 90, ha='center', rotation_mode='default')

    # Loop over data dimensions and create tet annotations
    for i in range(len(contacts.index)):
        for j in range(len(contacts.columns)):
            text = ax[a].text(j,i, contact_matrix[i,j], ha='center', va='center', color='w', fontsize = 5)
    fig.tight_layout()

In [ ]:
import nglview as nv

t = nv.MDAnalysisTrajectory(uni)
w = nv.NGLWidget(t)
w

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

vegetables = ["cucumber", "tomato", "lettuce", "asparagus",
              "potato", "wheat", "barley"]
farmers = ["Farmer Joe", "Upland Bros.", "Smith Gardening",
           "Agrifun", "Organiculture", "BioGoods Ltd.", "Cornylee Corp."]

harvest = np.array([[0.8, 2.4, 2.5, 3.9, 0.0, 4.0, 0.0],
                    [2.4, 0.0, 4.0, 1.0, 2.7, 0.0, 0.0],
                    [1.1, 2.4, 0.8, 4.3, 1.9, 4.4, 0.0],
                    [0.6, 0.0, 0.3, 0.0, 3.1, 0.0, 0.0],
                    [0.7, 1.7, 0.6, 2.6, 2.2, 6.2, 0.0],
                    [1.3, 1.2, 0.0, 0.0, 0.0, 3.2, 5.1],
                    [0.1, 2.0, 0.0, 1.4, 0.0, 1.9, 6.3]])


fig, ax = plt.subplots()
im = ax.imshow(harvest)

# We want to show all ticks...
ax.set_xticks(np.arange(len(farmers)))
ax.set_yticks(np.arange(len(vegetables)))
# ... and label them with the respective list entries
ax.set_xticklabels(farmers)
ax.set_yticklabels(vegetables)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(vegetables)):
    for j in range(len(farmers)):
        text = ax.text(j, i, harvest[i, j],
                       ha="center", va="center", color="w")

ax.set_title("Harvest of local farmers (in tons/year)")
fig.tight_layout()
plt.show()